In [ ]:
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

In [12]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [17]:
def expand_greyscale_image_channels(grey_pil_image):
    grey_image_arr = np.array(grey_pil_image)
    grey_image_arr = np.expand_dims(grey_image_arr, -1)
    grey_image_arr_3_channel = grey_image_arr.repeat(3, axis=-1)
    return grey_image_arr_3_channel

def preprocess_mnist(images: np.array):
    # reshape and upsample to 3 channel for transfer learning models
    images = expand_greyscale_image_channels(images)
    # normalize pixel values
    images = images.astype('float32') / 255.0
    # resize with pad for mobilenetv2
    images = tf.image.resize_with_pad(images, target_height=224, target_width=224)
    return images

x_train_processed = preprocess_mnist(x_train)
CLASS_NAMES = np.unique(y_train)

In [32]:
batch_size = 32
lrate = 0.001
l1 = 0.
l2 = 0.
num_hidden = 16
regularizer = tf.keras.regularizers.l1_l2(l1, l2)

layers = [
    hub.KerasLayer(
        "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4",
        input_shape=tuple(x_train_processed.shape[1:]),
        trainable=False,
        name='mobilenet_embedding'),
    tf.keras.layers.Dense(num_hidden,
                          kernel_regularizer=regularizer,
                          activation='relu',
                          name='dense_hidden'),
    tf.keras.layers.Dense(len(CLASS_NAMES),
                          kernel_regularizer=regularizer,
                          activation='softmax',
                          name='mnist_prob')
    ]
model = tf.keras.Sequential(layers, name='mnist_classification')
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lrate),
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(
                          from_logits=False),
                      metrics=['accuracy'])
model.fit(x_train_processed, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 590s 313ms/step - loss: 0.2848 - accuracy: 0.9177
Epoch 2/5
1875/1875 [==============================] - 606s 323ms/step - loss: 0.1023 - accuracy: 0.9692
Epoch 3/5
1875/1875 [==============================] - 600s 320ms/step - loss: 0.0776 - accuracy: 0.9757
Epoch 4/5
1875/1875 [==============================] - 686s 366ms/step - loss: 0.0657 - accuracy: 0.9798
Epoch 5/5
1875/1875 [==============================] - 551s 294ms/step - loss: 0.0553 - accuracy: 0.9828


In [35]:
from pathlib import Path
model_path = Path(__file__).parent / "model.joblib"

NameError: name '__file__' is not defined

In [ ]:
x_test_processed = preprocess_mnist(x_test)
# evaluate the model
loss, acc = model.evaluate(x_test_processed, y_test, verbose=0)
print('Accuracy: %.3f' % acc)

In [39]:
# model_path = Path(__file__).parent / "model.joblib"
#save model
ts = int(time.time())
file_path = f"./img_classifier/{ts}/"
model.save(filepath=file_path, save_format='tf')

INFO:tensorflow:Assets written to: ./img_classifier/1678311355/assets


INFO:tensorflow:Assets written to: ./img_classifier/1678311355/assets


In [51]:
import grpc
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
from tensorboard.compat.proto import types_pb2

channel = grpc.insecure_channel('127.0.0.1:8500')
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

request = predict_pb2.PredictRequest()
request.model_spec.name = 'mnist-model'
request.inputs['flatten_input'].CopyFrom(tf.make_tensor_proto(X_test[0],dtype=types_pb2.DT_FLOAT,  shape=[28,28,1]))